## A prototype for testing letter reconstruction
## under different hyper parameters

In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('Utils/')
from Inverse_Util import inverse_obj, draw_this, inv_error
from Image_Util import img2arr, noise, res2heatmap, getAnimation
from Inverse_CG_Util import CG_FEM_solver, CG_ROM_solver
import numpy as np
import pandas as pd
import matplotlib as plt
from ngsolve import *
from ngsolve.webgui import Draw
import time

plt.rc('xtick', labelsize=14)  
plt.rc('ytick', labelsize=14)
%reload_ext autoreload


## CG Test for Letter Source Term
- $u_0 = 0$
- Initial guess $f_0 = Sin(\pi x) \cdot Sin(\pi y)$
- Regularization $\lambda = 10^{-6}$
- Tolerance = $10^{-6}$
- h = 1/32, 1/64, 1/128, 1/256, 1/512
- $\Delta t$ = h
- T = 1
- noise on observation = *Gaussian($0, 10^{-3}$)* , level = 10%

In [ ]:
nn = [2**5, 2**6, 2**7, 2**8, 2**9]
the_lam = 1e-7
the_tol = 1e-7
lams = [1e-4,1e-5,1e-6,1e-7,1e-8]
tols = [1e-6,1e-7,1e-8,1e-9,1e-10]

cg_fem = dict(); cg_rom = dict(); cg_all = dict()
colnames =  ['h','lam','tol','ite','time','err']
colnames_all =  ['h','lam','tol','ite_f','ite_r','time_f','time_r',
                 'err_f','err_r','err_comp']
cols = [np.zeros((len(nn),len(colnames))) for i in range(2)]
cols_all = [np.zeros((len(nn),len(colnames))) for i in range(1)]

In [ ]:
idx = 2
the_h = 1/nn[idx]

path = "Letters/CMU3{}.png".format(nn[idx])
Fix = 3

#exact letter source input
sou = img2arr(path, checkGray = False, checkBw = False, sqxN = Fix)
#f0 = img2arr(path, checkGray = False, checkBw = False)
f0 = sin(pi*x)*sin(pi*y)

#struct for forward solving
X_fem = inverse_obj(h = the_h, order = 1, dim = 1, boundary = "bottom|left|right|top", 
                T = 1, dt = 1/32, u0 = 0, source = sou, sqr = True)
X_fem.get_mesh(sqxN=Fix)
out_tmp = X_fem.solve_fem(save = True, anim = True)
## for sol animation
out = noise(out_tmp[:,-1],level = 0.1, std = 1e-3)
#out = noise(out_tmp,level = 0.1, std = 1e-3)
X_fem.fem_uT = out

#displays the exact source
#draw_this(X_fem, X_fem.sou)
res2heatmap(sou, nn[idx], Fix*nn[idx], sqxN=Fix)

In [ ]:
%matplotlib notebook
## change fem_back_solver to output solution at all time steps
getAnimation(out_tmp, nn[idx], Fix*nn[idx], sqxN = Fix)

In [ ]:
#displays the final time obs
#draw_this(X_fem, out)
res2heatmap(X_fem.fem_uT, nn[idx], Fix*nn[idx], sqxN = Fix)

### FEM Test only

In [ ]:
ks = []; tms = []; errs = []

for i in range(1):
    res_fem, k, tm = CG_FEM_solver(X_fem, f0, lam = the_lam,
                                   tol2 = the_tol, ite = True, ite_num = 0, 
                                   anim = True)
    err = inv_error(X_fem, sou, res_fem[-1])
    ks.append(k)
    tms.append(tm)
    errs.append(err)
    
k0 = np.mean(ks); tm0 = np.mean(tms); err0 = np.mean(errs)
pack0 = [the_h,the_lam,the_tol,k0,tm0,err0]
cols[0][idx, :] = np.array(pack0)

#displays reconstruction
#draw_this(X_fem, res_fem)
res2heatmap(res_fem[-1], nn[idx], nn[idx])
print ("\nCG-FEM L2-error:", err0)
#getAnimation(res_fem, nn[idx], nn[idx], fix = True)

In [ ]:
a = []
for res in res_fem:
    a.append(inv_error(X_fem, sou, res))
print("\nmin CG-FEM L2-error:", min(a))
getAnimation(res_fem, nn[idx], nn[idx], fix = True)

### ROM Test only

In [ ]:
ks = []; tms = []; errs = []

for i in range(1):
    res_rom, k, tm = CG_ROM_solver(X_fem, f0, lam = the_lam,
                                  tol2 = the_tol, ite = True, anim = True)
    err = inv_error(X_fem, sou, res_rom[-1])
    ks.append(k)
    tms.append(tm)
    errs.append(err)
    
k1 = np.mean(ks); tm1 = np.mean(tms); err1 = np.mean(errs)
pack1 = [the_h,the_lam,the_tol,k1,tm1,err1]
cols[1][idx, :] = np.array(pack1)

#displays reconstruction
#draw_this(X_fem, res_rom)
res2heatmap(res_rom[-1], nn[idx], nn[idx])
print ("\nCG-ROM L2-error:", err1)

In [ ]:
a = []
for res in res_rom:
    a.append(inv_error(X_fem, sou, res))
print("\nmin CG-FEM L2-error:", min(a))
getAnimation(res_rom, nn[idx], nn[idx], fix = True)

# Wait until $idx = 5$

In [ ]:
for i in range(6):
    cg_fem[colnames[i]] = cols[0][:,i]
    cg_rom[colnames[i]] = cols[1][:,i]
cg_fem_df = pd.DataFrame(data = cg_fem)
cg_rom_df = pd.DataFrame(data = cg_rom)
cg_fem_df.to_csv('CMUcg_fem_df.csv')
cg_rom_df.to_csv('CMUcg_rom_df.csv')

## FEM+ROM Tests

In [ ]:
ks_f = []; tms_f = []; errs_f = []
ks_r = []; tms_r = []; errs_r = []
errs_c = []

for i in range(3):
    res_fem, k_f, tm_f = CG_FEM_solver(X_fem, f0, lam = the_lam,
                                  tol2 = the_tol, ite = True, ite_num = 0)
    res_rom, k_r, tm_r = CG_ROM_solver(X_fem, f0, lam = the_lam, withM = True, 
                                  tol2 = the_tol, ite = True)
    
    err_fem = inv_error(X_fem, sou, res_fem)
    err_rom = inv_error(X_fem, sou, res_rom)
    err_comp = inv_error(X_fem, res_fem, res_rom)
    
    ks_f.append(k_f)
    tms_f.append(tm_f)
    errs_f.append(err_fem)
    
    ks_r.append(k_r)
    tm_r.append(tm_r)
    errs_r.append(err_rom)
    errs_c.append(err_comp)
    
err01_c = np.mean(errs_c)
k0 = np.mean(ks_f); tm0 = np.mean(tms_f); err0 = np.mean(errs_f)
k1 = np.mean(ks_f); tm1 = np.mean(tms_f); err1 = np.mean(errs_r)
pack_all = [the_h,the_lam,the_tol,k0,k1,tm0,tm1,err0,err1,err01_c]
cols_all[0][idx, :] = np.array(pack_all)

#displays reconstruction
#draw_this(X_fem, res_fem)
#res2heatmap(res_fem, nn[idx], nn[idx])
print ("\nCG-FEM L2-error:", err0)
print ("\nCG-ROM L2-error:", err1)

In [ ]:
for i in range(6):
    cg_all[colnames_all[i]] = cols_all[0][:,i]
    #cg_rom[colnames[i]] = cols[1][:,i]
cg_all_df = pd.DataFrame(data = cg_all)
cg_all_df.to_csv('CMU3cg_all_df.csv')


In [ ]:
cg_fem_df

In [ ]:
cg_rom_df

In [ ]:
diff = inv_error(X_fem, res_fem, res_rom)
print(diff)

In [ ]:
# res_bf_fem = BFGS_FEM_solver(X_fem, f0, 1, lam = 1e-6, tol2 = 1e-6, 
#                                       ite = False, ite_num = 500)

# #displays reconstruction
# draw_this(X_fem, res_bf_fem)
# err = inv_error(X_fem, sou, res_bf_fem)
# print ("\nBFGS_fem L2-error:", err)